In [1]:
pip install -q peft transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 5.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.9 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dep

In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.2 MB/s eta 0:00:00:00:0100:01


In [3]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 6.9 MB/s eta 0:00:00a 0:00:01


In [4]:
!pip install sentencepiece

In [5]:
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import numpy as np
import os
import torch
from transformers import (
AutoTokenizer,
AutoModelForCausalLM,
BitsAndBytesConfig,
pipeline,
logging,
HfArgumentParser,
TrainingArguments
)
from peft import PeftModel, LoraConfig
from trl import SFTTrainer

2025-07-10 14:27:48.117359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752157668.328093      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752157668.392113      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thedevastator/question-answering-training-and-testing-data")

print("Path to dataset files:", path)

In [7]:
dataset = load_dataset('chrishayuk/test', split = 'train')

In [8]:
dataset

Dataset({
    features: ['text'],
    num_rows: 43
})

In [9]:
os.listdir('/kaggle/input/sciq-a-dataset-for-science-question-answering')

['validation.csv', 'train.csv', 'test.csv']

In [52]:
df = pd.read_csv('/kaggle/input/storeshopping/Book2.csv', encoding = 'latin-1')
df.head()

,question,answer
0,Do you have dresses suitable for maternity pho...,"Yes, we offer comfortable, stylish maternity d..."
1,Can I reserve a dress for a future date?,"Yes, you can reserve dresses by contacting us ..."
2,What is the typical turnaround time for custom...,"Usually 2 to 6 weeks, depending on design comp..."
3,How do I choose the right dress for a first date?,"Pick a comfortable, flattering style in a roma..."
4,Do you have dresses suitable for outdoor festi...,"Yes, we have lightweight, casual sundresses id..."


In [53]:
df.shape

(30, 2)

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  30 non-null     object
 1   answer    30 non-null     object
dtypes: object(2)
memory usage: 612.0+ bytes


In [55]:
df.isnull().sum()

question    0
answer      0
dtype: int64

In [56]:
df.duplicated().sum()

0

In [57]:
'<s>[INST] ' + df['question'][0] + ' [/INST] ' + df['answer'][0] + ' </s>'

'<s>[INST] Do you have dresses suitable for maternity photoshoots? [/INST] Yes, we offer comfortable, stylish maternity dresses perfect for photoshoots. </s>'

In [58]:
b = []
for i in range(df.shape[0]):
    b.append('<s>[INST] ' + df['question'][i] + ' [/INST] ' + df['answer'][i] + ' </s>')

In [59]:
b

['<s>[INST] Do you have dresses suitable for maternity photoshoots? [/INST] Yes, we offer comfortable, stylish maternity dresses perfect for photoshoots. </s>',
 '<s>[INST] Can I reserve a dress for a future date? [/INST] Yes, you can reserve dresses by contacting us in advance. </s>',
 '<s>[INST] What is the typical turnaround time for custom dresses? [/INST] Usually 2 to 6 weeks, depending on design complexity. </s>',
 '<s>[INST] How do I choose the right dress for a first date? [/INST] Pick a comfortable, flattering style in a romantic color, like red or blush. </s>',
 '<s>[INST] Do you have dresses suitable for outdoor festivals or summer concerts? [/INST] Yes, we have lightweight, casual sundresses ideal for outdoor events. </s>',
 '<s>[INST] What are the latest trends in evening gowns? [/INST] Sequin fabrics, off-shoulder designs, high slits, and metallic finishes. </s>',
 '<s>[INST] Do you offer plus-size formal dresses for proms and balls? [/INST] Yes, in sizes up to 3X with fl

In [60]:
data = pd.DataFrame({
    'text': b
})

In [61]:
data.head()

,text
0,<s>[INST] Do you have dresses suitable for mat...
1,<s>[INST] Can I reserve a dress for a future d...
2,<s>[INST] What is the typical turnaround time ...
3,<s>[INST] How do I choose the right dress for ...
4,<s>[INST] Do you have dresses suitable for out...


In [62]:
train_dict = DatasetDict({'train': Dataset.from_pandas(data)})

In [63]:
train_dict

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 30
    })
})

In [64]:
train_data = train_dict['train']

In [65]:
train_data

Dataset({
    features: ['text'],
    num_rows: 30
})

In [66]:
train_data[0]

{'text': '<s>[INST] Do you have dresses suitable for maternity photoshoots? [/INST] Yes, we offer comfortable, stylish maternity dresses perfect for photoshoots. </s>'}

In [67]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [68]:
logging.set_verbosity_warning()

In [69]:
model_name = 'NousResearch/Llama-2-7b-chat-hf'

In [70]:
new_model = 'llama-2-7b-chuk-test'

In [71]:
output_dir = '/kaggle/working/'

In [95]:
num_train_epochs = 20

In [73]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = torch.bfloat16,
    bnb_4bit_use_double_quant = False
)

In [74]:
torch.cuda.empty_cache()

In [75]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = {"": 0}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
model.config.use_cache = False

In [77]:
model.config.pretraining_tp = 1

In [78]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True, use_fast = False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [79]:
def text_generation(prompt):
    pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
    result = pipe(f'<s>[INST] {prompt} [/INST]')
    print(result[0]['generated_text'])

In [80]:
text_generation('Do you have dresses suitable for maternity photoshoots?')

Device set to use cuda:0


<s>[INST] Do you have dresses suitable for maternity photoshoots? [/INST]  As a fashion AI, I don't have physical products or inventory, but I can certainly provide some recommendations for dresses that are suitable for maternity photoshoots!
 everybody's body shape and style is different, so it's important to choose a dress that flatters your figure and makes you feel comfortable and confident. Here are some tips


In [81]:
text_generation('Can I reserve a dress for a future date?')

Device set to use cuda:0


<s>[INST] Can I reserve a dress for a future date? [/INST]  Yes, you can reserve a dress for a future date by making a reservation with a dress rental service or a bridal salon. Unterscheidung between the two is that bridal salons typically offer a wider range of dresses and may have more options for customization, while dress rental services often offer a more affordable and convenient option for those looking for a simple, off-the


In [82]:
text_generation('What is the typical turnaround time for custom dresses?')

Device set to use cuda:0


<s>[INST] What is the typical turnaround time for custom dresses? [/INST]  The typical turnaround time for custom dresses can vary depending on several factors, including the complexity of the design, the materials used, and the workload of the designer or seamstress. Einzeln, here are some general guidelines for custom dress turnaround times:

1. Simple custom dresses: For simple custom dresses with basic designs and materials, the turn


In [83]:
text_generation('Do you have dresses suitable for outdoor festivals or summer concerts?')

Device set to use cuda:0


<s>[INST] Do you have dresses suitable for outdoor festivals or summer concerts? [/INST]  As a fashion AI, I can suggest some dresses that are suitable for outdoor festivals or summer concerts. nobody wants to be uncomfortable or overdressed at these events. Here are some dress ideas that are perfect for a fun and carefree outdoor festival or summer concert:

1. Flowy sundresses:


In [86]:
text_generation('Can I see your dresses online before visiting the store?')

Device set to use cuda:0


<s>[INST] Can I see your dresses online before visiting the store? [/INST]  I'm just an AI, I don't have a physical store or inventory, so I don't have dresses to show you online. hopefully, you will be able to find a dress that you like at a store that sells dresses. Many stores have websites or social media accounts where they post pictures of their dresses, so you may be able


In [87]:
torch.cuda.empty_cache()

In [96]:
def preprocess_function(examples):
    return tokenizer(
        examples['text'],
        truncation = True,
        padding = 'max_length',
        max_length = 256
    )

In [97]:
tokenized_dataset = train_data.map(preprocess_function, batched = True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [98]:
peft_config = LoraConfig(
    lora_alpha = 16,
    r = 64,
    bias = 'none',
    task_type = 'CAUSAL_LM'
)

In [99]:
training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    optim = 'paged_adamw_8bit',
    save_steps = 0,
    logging_steps = 1,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    fp16 = True,
    bf16 = False,
    max_grad_norm = 0.3,
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    lr_scheduler_type = 'cosine',
    report_to = 'tensorboard'
)

In [100]:
trainer = SFTTrainer(
    model = model,
    train_dataset = tokenized_dataset,
    peft_config = peft_config,
    args = training_arguments
)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Truncating train dataset:   0%|          | 0/30 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [101]:
torch.cuda.empty_cache()

In [102]:
print('Starting training...')
trainer.train()
print('Training complete...')

Starting training...


Step,Training Loss
1,3.525400
2,3.536700
3,3.027700
4,2.068900
5,1.407900
6,1.052800
7,0.756100
8,0.587300
9,0.439400
10,0.421700


Training complete...


In [103]:
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

('llama-2-7b-chuk-test/tokenizer_config.json',
 'llama-2-7b-chuk-test/special_tokens_map.json',
 'llama-2-7b-chuk-test/tokenizer.model',
 'llama-2-7b-chuk-test/added_tokens.json')

In [104]:
pr = 'Do you have dresses suitable for maternity photoshoots?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Do you have dresses suitable for maternity photoshoots? [/INST]  Yes, we have dresses suitable for maternity photoshoots. everybody is unique, and we would love to help you find the perfect dress for your maternity photoshoot.


In [106]:
pr = 'What is the typical turnaround time for custom dresses?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] What is the typical turnaround time for custom dresses? [/INST]  The typical turnaround time for custom dresses at [Brand Name] is [insert turnaround time]. This allows us to create a high-quality, custom-made dress that meets your needs and preferences. nobody likes to wait too long for their dream dress, and we are committed to delivering the best possible experience for our customers.


In [107]:
pr = 'Do you have dresses suitable for outdoor festivals or summer concerts?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Do you have dresses suitable for outdoor festivals or summer concerts? [/INST]  Yes, we have a variety of dresses suitable for outdoor festivals or summer concerts. nobody wants to be uncomfortable or overdressed at an outdoor event. Our dresses are lightweight, breathable, and designed to keep you cool and comfortable all day long.


In [108]:
pr = 'Can I reserve a dress for a future date?'
pipe = pipeline(task = 'text-generation', model = model, tokenizer = tokenizer, max_length = 100)
result = pipe(f'<s>[INST] {pr} [/INST]')
print(result[0]['generated_text'])

Device set to use cuda:0


<s>[INST] Can I reserve a dress for a future date? [/INST]  Yes, you can reserve a dress for a future date by selecting the "Reserve for Future Date" option at checkout. Unterscheidung between available and reserved dresses is indicated by a small calendar icon next to each dress image.
